In [ ]:
import ixmp
import yaml
import pyam
import nomenclature

In [ ]:
ar6_public = ixmp.Platform("ar6")
ar6_submission = ixmp.Platform("ar6-submission")

In [ ]:
ar6_public_scenario_list = ar6_public.scenario_list()
ar6_submission_scenario_list = ar6_submission.scenario_list()

In [ ]:
with open("region_rename.yaml") as f:
    region_rename_mapping = yaml.safe_load(f)

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
ssp_scenarios = (
    [f"SSP1-{x}" for x in ["19", "26", "34", "45", "Baseline"]] +
    [f"SSP2-{x}" for x in ["19", "26", "34", "45", "60", "Baseline"]] +
    [f"SSP3-{x}" for x in ["34", "45", "60", "Baseline"]] +
    [f"SSP4-{x}" for x in ["26", "34", "45", "Baseline"]] +
    [f"SSP5-{x}" for x in ["26", "34", "45", "60", "Baseline"]]
)

_scenarios = ar6_public_scenario_list[[s in ssp_scenarios for s in ar6_public_scenario_list.scenario]]

In [ ]:
ar6_submission_scenarios = ar6_submission_scenario_list[[s in ssp_scenarios for s in ar6_submission_scenario_list.scenario]]

In [ ]:
scenario_selection = list()
for i, row in _scenarios.iterrows():
    scenario_selection.append((row["model"], row["scenario"]))

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=ar6_submission, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.rename(region=region_rename_mapping, inplace=True)

In [ ]:
df.rename(region={
    "R5OECD90+EU": "OECD & EU (R5)",
    "R5ASIA": "Asia (R5)",
    "R5REF": "Reforming Economies (R5)",
    "R5MAF": "Middle East & Africa (R5)",
    "R5LAM": "Latin America (R5)",
}, inplace=True)

In [ ]:
df.region

In [ ]:
df.filter(region=[
    "China", "India", "Japan", "Brazil", "Mexico", 
    "United States of America", "European Union (28 member countries)",
    "GCAM*"], keep=False, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)
    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the IPCC AR6 database")
    scen.set_as_default()

    scen.set_meta({"Source": "SSP (via IPCC AR6)"})
    
    print(f"Model {_model} - Scenario {_scenario}" )